In [1]:
import requests
import pandas as pd
from dotenv import load_dotenv
load_dotenv('../05_data_scraping/.env')
import os
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import koreanize_matplotlib
import time
from konlpy.tag import Mecab
mecab = Mecab()
from nltk.probability import FreqDist
from wordcloud import WordCloud
import re
from datetime import datetime

### 함수 만들기

In [4]:
def string_delete(text) :
    return text.replace("<b>","").replace("</b>", "").replace("&quot;", "")

def parse_date(date_str):
    # 날짜 문자열을 datetime 객체로 변환
    return datetime.strptime(date_str, "%Y%m%d")

def make_df(df_name) :
    for i in df_name :
        for idx in range(0, len(df_name[i])) : 
            df_name[i][idx] = string_delete(df_name[i][idx])
    df_name['postdate'] = df_name['postdate'].apply(parse_date)
    return df_name

In [45]:
def w2v(keyword) :
    client_id = os.getenv('client_id') # 환경 변수에 접근
    client_pw = os.getenv('client_pw') # 환경 변수에 접근

    stopwords = pd.read_csv('https://raw.githubusercontent.com/haram4th/ablearn/main/%ED%95%9C%EA%B5%AD%EC%96%B4%EB%B6%88%EC%9A%A9%EC%96%B4100.txt', header = None)
    split_keyword = keyword.split(' ')
    ex_stop_words = [f'{split_keyword[0]}역']
    for i in ex_stop_words : 
        stopwords = stopwords.drop(stopwords[stopwords[0] == i].index)
    stopwords.reset_index(drop=True, inplace=True)
    
    url = "https://openapi.naver.com/v1/search/blog.json"
    final_data_keyword = pd.DataFrame()
    start = time.time() # 시작
    for i in range(1, 1001) :
        payload = {'query' : keyword, 'display' : 100, 'start' : i, 'sort' : 'sim'}
        headers = {'X-Naver-Client-Id' : client_id, 'X-Naver-Client-Secret' : client_pw}
        r = requests.get(url, params = payload, headers = headers)
        if r.status_code == 200 :
            data = r.json()
        else :
            print("Error Code:", r.status_code)
            
        df_keyword_sim = pd.json_normalize(data['items'])
        entire_data_keyword_sim = make_df(df_keyword_sim)
        final_data_keyword = pd.concat([final_data_keyword, entire_data_keyword_sim], ignore_index=True)
        # 형태소 분석 후 후처리
    def process_title(title, keyword):
        # '망리단길'을 임시 토큰으로 변경
        split_keyword = keyword.split(' ')
        title = title.replace(split_keyword[0], '우주')
        # 형태소 분석
        words = mecab.pos(re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", title))
        # 임시 토큰을 원래의 '망리단길'로 복원
        processed_words = [(word[0].replace('우주', keyword), word[1]) for word in words]
        return processed_words
        
    # DataFrame에 적용
    final_data_keyword['title2'] = final_data_keyword['title'].apply(lambda x : process_title(x, keyword))

    target = ['NNG', 'NNP', 'MAG', 'XR', 'NR', 'VV', 'VA', 'VCP', 'VCN', 'MM', 'IC', 'XPN', 'UNKNOWN']

    # 형태소에서 특정 단어 제거 및 태그 필터링
    def extract_words(words_list) : 
        return [word for word, pos in words_list if pos in target]
    final_data_keyword['filtered'] = final_data_keyword['title2'].apply(lambda x : extract_words(x))
    final_data_keyword['filtered_list'] = final_data_keyword['filtered'].apply(lambda x : eval(x) if isinstance(x, str) else x)
    from gensim.models import Word2Vec
    model_keyword_pos = Word2Vec(final_data_keyword['filtered_list'], vector_size=100, window=5, min_count=2, workers=8, sg=0)
    model_keyword_pos_result = model_keyword_pos.wv.most_similar(keyword, topn = 50)
    # 워드투벡터 결과 소수점 4자리로 변경하기
    w2v_result = list((item[0], round(item[1], 4)) for item in model_keyword_pos_result)
    print(f"{time.time()-start:.4f} sec")
    return keyword, w2v_result

### 망리단길 오복수산시장 가져와보기

In [9]:
obok = w2v('오복수산시장 망원')
obok

1.9888 sec


('오복수산시장 망원',
 [('망원', 0.9990739822387695),
  ('시장', 0.9990715384483337),
  ('횟집', 0.9990391135215759),
  ('즐기', 0.9990267753601074),
  ('복덕방', 0.9989977478981018),
  ('모듬', 0.9989476203918457),
  ('먹', 0.998908281326294),
  ('차', 0.9988926649093628),
  ('방문', 0.9988802671432495),
  ('연남', 0.9988481998443604),
  ('오복', 0.9988437294960022),
  ('해산물', 0.9988002777099609),
  ('수산', 0.9987565875053406),
  ('홍대', 0.9987555146217346),
  ('한강', 0.9987309575080872),
  ('술집', 0.9986664056777954),
  ('사시미', 0.9986526370048523),
  ('방어', 0.9986301064491272),
  ('좋', 0.9985864162445068),
  ('공원', 0.9985700845718384),
  ('갑오징어', 0.9985272288322449),
  ('망원역', 0.998526930809021),
  ('망원동', 0.9985177516937256),
  ('공작', 0.9985009431838989),
  ('파티', 0.9984629154205322),
  ('맛', 0.9984500408172607),
  ('물회', 0.9984319806098938),
  ('닭', 0.998428225517273),
  ('맛집', 0.9984257221221924),
  ('추천', 0.998408317565918),
  ('섭식', 0.9984010457992554),
  ('길', 0.9983846545219421),
  ('주담', 0.9983789920806885),

### 다미만두 망원점 가져와보기

In [8]:
dami = w2v('다미만두 망원점')
dami

1.0303 sec


('다미만두 망원점',
 [('만둣국', 0.28473061323165894),
  ('후기', 0.26690784096717834),
  ('서교동', 0.2120930403470993),
  ('향수', 0.20405590534210205),
  ('아시나', 0.20261718332767487),
  ('다', 0.19577562808990479),
  ('망원', 0.1749344915151596),
  ('음식점', 0.16782274842262268),
  ('조용', 0.16457249224185944),
  ('다미', 0.1639460325241089),
  ('새로', 0.16151641309261322),
  ('없이', 0.1607184112071991),
  ('매콤', 0.15655799210071564),
  ('형', 0.14693596959114075),
  ('동네', 0.138671875),
  ('곳', 0.12090960144996643),
  ('면허', 0.11929824948310852),
  ('청기', 0.11293929815292358),
  ('음식', 0.10364046692848206),
  ('가게', 0.1002303808927536),
  ('방송', 0.09438923001289368),
  ('받', 0.08855488896369934),
  ('특허', 0.08783376216888428),
  ('유성운수', 0.08307520300149918),
  ('홈파티', 0.07872135192155838),
  ('앞', 0.07482094317674637),
  ('김치', 0.07389651238918304),
  ('서울', 0.07072798907756805),
  ('감성', 0.06809695810079575),
  ('두', 0.06633927673101425),
  ('입', 0.06242230907082558),
  ('노스터', 0.05182337015867233),
  ('미만'

### 밀면집 망원 가져와보기

In [12]:
idiya = w2v('밀면집 망원')
idiya

2.1382 sec


('밀면집 망원',
 [('밀면', 0.9992455840110779),
  ('집', 0.9990858435630798),
  ('북', 0.9990081787109375),
  ('망원동', 0.998977541923523),
  ('마인드', 0.9989655613899231),
  ('비', 0.9989597797393799),
  ('망원', 0.9989415407180786),
  ('맛집', 0.9989396929740906),
  ('한', 0.9988927841186523),
  ('밀', 0.9988836050033569),
  ('삼', 0.9988521933555603),
  ('팁', 0.9988447427749634),
  ('주차', 0.9988359212875366),
  ('방문', 0.9987961649894714),
  ('언더', 0.9987891912460327),
  ('핫', 0.9987813830375671),
  ('애', 0.9987689256668091),
  ('서울', 0.998748779296875),
  ('차장', 0.9987356662750244),
  ('할인', 0.9986929297447205),
  ('카스테라', 0.9986885190010071),
  ('최고', 0.9986881017684937),
  ('시장', 0.9986796379089355),
  ('먹', 0.9986502528190613),
  ('후기', 0.998617947101593),
  ('부산', 0.9985747933387756),
  ('동', 0.9985684752464294),
  ('맛', 0.9985681772232056),
  ('한강', 0.9985425472259521),
  ('줄', 0.9985341429710388),
  ('추가', 0.9985180497169495),
  ('여름', 0.9985055923461914),
  ('공영주', 0.9984226226806641),
  ('공원', 0

### 토리키치 망원점 가져와보기

In [44]:
tori = w2v('토리키치 망원')
tori

                                     title  \
0        토리키치 망원 이차로 가기... 튀김 분위기 좋은 망원동술집   
1       망원 술집 ‘토리키치 망원점’ 데이트하기 좋은 망원동 이자카야   
2       서울, 망원 : 토리키치 | 망원동 술집 야키토리 하이볼 맛집   
3    망원에 드디어 오픈한 꼬치맛집 술집 토리키치 망원점 | 특선...    
4                        망원 토리키치 야키토리 이자카야   
..                                     ...   
121                     merry christmas 끝~   
122                    중요한건 꺾였는데도 그냥 하는 마음   
123                                  9월 기록   
124                               5, 6월 일상   
125            너조차 널 미워한 밤이면 내가 너의 편이 되어줄게   

                                                link  \
0       https://blog.naver.com/joo_0798/223603712890   
1       https://blog.naver.com/tnwl6906/223552413808   
2     https://blog.naver.com/qordms0124/223548663933   
3     https://blog.naver.com/heeejjaang/223543465603   
4      https://blog.naver.com/hongkhmii/223605386123   
..                                               ...   
121     https://blog.naver.com/mon_dol_/222967189277   

('토리키치 망원',
 [('느낌', 0.1999),
  ('갑', 0.1958),
  ('포스팅', 0.1947),
  ('식당', 0.1943),
  ('키', 0.1911),
  ('계절', 0.1771),
  ('분위기', 0.1714),
  ('벌써', 0.163),
  ('서울', 0.1541),
  ('완연', 0.142),
  ('자카야', 0.1391),
  ('때', 0.1331),
  ('끝', 0.123),
  ('봄', 0.1198),
  ('카츠', 0.1118),
  ('집', 0.1101),
  ('하우스', 0.0938),
  ('요', 0.0932),
  ('마음', 0.0907),
  ('카츠야', 0.0834),
  ('망원동', 0.0773),
  ('비로소', 0.0768),
  ('첫째', 0.0721),
  ('특선', 0.0687),
  ('촉', 0.0674),
  ('고양이', 0.0673),
  ('기억', 0.0638),
  ('함께', 0.0624),
  ('참으로', 0.06),
  ('주민', 0.0579),
  ('야키', 0.0557),
  ('앵', 0.0512),
  ('추천', 0.0444),
  ('음식점', 0.0441),
  ('쿠시', 0.0417),
  ('나무', 0.0416),
  ('속', 0.041),
  ('마지막', 0.0388),
  ('모든', 0.036),
  ('토리', 0.0354),
  ('오픈', 0.0348),
  ('조용', 0.0341),
  ('망원', 0.0291),
  ('기획', 0.0246),
  ('신상', 0.0229),
  ('술집', 0.0221),
  ('소소', 0.018),
  ('공덕', 0.0088),
  ('에', 0.0085),
  ('좋', 0.0058)])

### 한지 망원 가져와보기

In [40]:
hanji = w2v('한지 망원')
hanji

[망원술집추천] 우주 - 망원한식요리주점(조용한술집추천)
[('망원', 'NNG'), ('술집', 'NNG'), ('추천', 'NNG'), ('우주', 'NNG'), ('망원', 'NNG'), ('한식', 'NNG'), ('요리', 'NNG'), ('주점', 'NNG'), ('조용', 'XR'), ('한', 'XSA+ETM'), ('술집', 'NNG'), ('추천', 'NNG')]
[망원 우주] 망원 술집중에 최고야.... 안주 맛있고 분위기 좋은... 
[('망원', 'NNG'), ('우주', 'NNG'), ('망원', 'NNG'), ('술집', 'NNG'), ('중', 'NNB'), ('에', 'JKB'), ('최고', 'NNG'), ('야', 'VCP+EF'), ('안주', 'MAG'), ('맛있', 'VA'), ('고', 'EC'), ('분위기', 'NNG'), ('좋', 'VA'), ('은', 'ETM')]
망원_한식주점 우주
[('망원', 'NNG'), ('한', 'XSA+ETM'), ('식', 'NNB'), ('주점', 'NNG'), ('우주', 'NNG')]
[망원 맛집 요리주점] 우주 - 복어탕, 숙회, 닭볶음탕
[('망원', 'NNG'), ('맛집', 'NNG'), ('요리', 'NNG'), ('주점', 'NNG'), ('우주', 'NNG'), ('복어', 'NNG'), ('탕', 'NNG'), ('숙회', 'NNG'), ('닭볶음탕', 'NNG')]
[망원] 요리주점 망원 '우주', 한식 안주가 맛있는 망원동 술집... 
[('망원', 'NNG'), ('요리', 'NNG'), ('주점', 'NNG'), ('망원', 'NNG'), ('우주', 'NNG'), ('한식', 'NNG'), ('안주', 'NNG'), ('가', 'JKS'), ('맛있', 'VA'), ('는', 'ETM'), ('망원동', 'NNP'), ('술집', 'NNG')]
[망원 우주] 망원 술집 망원 술집 추천 숙취해소엔 부스틱
[('망원', 'NNG'), ('우주', 'NN

('한지 망원',
 [('망원', 0.8704),
  ('맛집', 0.8053),
  ('시장', 0.7853),
  ('일', 0.783),
  ('펌', 0.7774),
  ('속눈썹', 0.7714),
  ('좋', 0.7659),
  ('레이스', 0.7636),
  ('수묵', 0.7613),
  ('술집', 0.7601),
  ('망리', 0.7595),
  ('망원동', 0.7581),
  ('월', 0.7543),
  ('카페', 0.749),
  ('서울', 0.7459),
  ('감성', 0.7376),
  ('한강', 0.7334),
  ('단', 0.731),
  ('본점', 0.7257),
  ('장', 0.7187),
  ('예쁘', 0.7177),
  ('주점', 0.7106),
  ('합정', 0.7018),
  ('뷰티플', 0.7002),
  ('블링', 0.6996),
  ('어', 0.6983),
  ('그림', 0.6946),
  ('다', 0.6923),
  ('문화', 0.6891),
  ('공원', 0.6831),
  ('동네', 0.6815),
  ('코스', 0.6798),
  ('일상', 0.6786),
  ('이', 0.6775),
  ('맛있', 0.6772),
  ('후기', 0.6757),
  ('집', 0.6739),
  ('먹그림', 0.6738),
  ('있', 0.6714),
  ('비', 0.6711),
  ('휘', 0.6708),
  ('낭시', 0.6599),
  ('길', 0.6574),
  ('방문기', 0.657),
  ('홍대', 0.6536),
  ('토마토', 0.6476),
  ('축제', 0.6466),
  ('구경', 0.645),
  ('되', 0.6391),
  ('닭강정', 0.639)])